### Agent-Lab: Multi-Agent -> Browser

Objective of this notebook is evaluating and adapting a [Multi-Agent Supervisor Architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor) with coordinator and execution planning steps.

---


In [1]:
%%capture
import json
import os
import nest_asyncio
from IPython.display import Markdown, display
from dotenv import load_dotenv
from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest


os.chdir("..")
load_dotenv()
nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

---
### Anthropic Browser

In [2]:
# Create Workflow
anthropic_agent = experiment_utils.create_anthropic_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="claude-sonnet-4-0", api_key=os.getenv("ANTHROPIC_API_KEY")
)
anthropic_browser_agent = container.coordinator_planner_supervisor_agent()
anthropic_workflow_builder = anthropic_browser_agent.get_workflow_builder(anthropic_agent["id"])
anthropic_workflow = anthropic_workflow_builder.compile(checkpointer=checkpointer)

In [3]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="Using web browser at your disposal, visit https://en.wikipedia.org/wiki/Mathematical_finance and rewrite the first paragraph in a simpler way that a 12 year old would understand.",
    agent_id=anthropic_agent["id"],
)

inputs = anthropic_browser_agent.get_input_params(message, schema="public")
config = anthropic_browser_agent.get_config(anthropic_agent["id"])
result = anthropic_workflow.invoke(inputs, config)
ai_message_content, workflow_state = anthropic_browser_agent.format_response(result)

In [4]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "The user wants me to visit a specific Wikipedia page about Mathematical Finance and rewrite the first paragraph in simpler language that a 12-year-old would understand. This requires me to: 1) Use the browser agent to visit the Wikipedia page and extract the first paragraph, 2) Then use the reporter agent to rewrite that content in age-appropriate, simple language.",
  "title": "Simplify Mathematical Finance Wikipedia Article for 12-Year-Olds",
  "steps": [
    {
      "agent_name": "browser",
      "title": "Visit Wikipedia page and extract first paragraph",
      "description": "Navigate to https://en.wikipedia.org/wiki/Mathematical_finance and locate the first paragraph of the article. Extract the complete text of the first paragraph to understand what mathematical finance is about. The browser agent should capture the exact content that needs to be simplified."
    },
    {
      "agent_name": "reporter",
      "title": "Rewrite content for 12-year-olds",
      "description": "Take the first paragraph content extracted from the Wikipedia page and rewrite it in simple, clear language that a 12-year-old can easily understand. Use everyday words, shorter sentences, and relatable examples. Avoid technical jargon and complex mathematical concepts. Present the simplified version as the final report."
    }
  ]
}
```

In [5]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

Perfect! Now I can rewrite this first paragraph in a way that a 12-year-old would understand:

**Here's the simplified version:**

**Mathematical finance** (also called quantitative finance or financial mathematics) is basically using math to help solve money problems. It's when smart people use numbers and calculations to figure out things about money and investing. There are two main ways people use this math with money: 

1. First, they use it to figure out how much certain financial products (called derivatives) should cost
2. Second, they use it to help people manage risk (which means avoiding losing money) and decide how to invest their money wisely

Think of it like this: just like you might use math to figure out if you have enough allowance money to buy something you want, financial mathematicians use much more complicated math to help banks, companies, and investors make smart decisions with much larger amounts of money!